In [1]:
!pip install weave openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.4/328.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.8/325.8 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 5.9 MB/s eta 0:00:00


In [2]:
import weave

weave.init("rag-baselining")

Please login to Weights & Biases (https://wandb.ai/) to continue:


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Logged in as Weights & Biases user: pratikbhande0.
View Weave data at https://wandb.ai/pratikbhande0-personal/rag-baselining/weave


In [3]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [6]:
from openai import OpenAI

client = OpenAI()
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Translate English to Hindi."},
        {"role": "user", "content": "What is ML?."},
    ],
    temperature=0,
)


generation = response.choices[0].message.content
print(generation)

🍩 https://wandb.ai/pratikbhande0-personal/rag-baselining/r/call/019416f2-ee82-7d11-bea3-48ae874a5929
ML का मतलब मशीन लर्निंग (Machine Learning) है। यह एक प्रकार की आर्टिफिशियल इंटेलिजेंस (Artificial Intelligence) है, जिसमें कंप्यूटर सिस्टम डेटा से सीखते हैं और अनुभव के आधार पर अपने प्रदर्शन में सुधार करते हैं, बिना किसी विशेष प्रोग्रामिंग के। मशीन लर्निंग का उपयोग विभिन्न क्षेत्रों में किया जाता है, जैसे कि डेटा एनालिसिस, छवि पहचान, प्राकृतिक भाषा प्रसंस्करण, और बहुत कुछ।


In [7]:
@weave.op()
def translation(user_input):
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Please translate English into Hindi."},
            {"role": "user", "content": user_input},
        ],
        temperature=0,
    )
    return response.choices[0].message.content

result = translation("AI is a old technology")
print(result)

🍩 https://wandb.ai/pratikbhande0-personal/rag-baselining/r/call/019416f3-361f-7c20-ab53-aa5dd4cef3b4
एआई एक पुरानी तकनीक है।


In [8]:
from openai import OpenAI

class TranslationModel(weave.Model):
    system_instruction: str

    @weave.op()
    def translation(self, user_input):
        client = OpenAI()
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": self.system_instruction},
                {"role": "user", "content": user_input},
            ],
            temperature=0,
        )
        return response.choices[0].message.content


model = TranslationModel(
    system_instruction="Please translate English into Hindi.",
)
result = model.translation("Generative AI has been hyped up a lot.")
print(result)

🍩 https://wandb.ai/pratikbhande0-personal/rag-baselining/r/call/019416f3-73d2-7102-b8f7-bb116205e85a
जनरेटिव एआई को बहुत अधिक प्रचारित किया गया है।


In [9]:
documents = [
    "SpaceX is a private aerospace manufacturer and space transportation company founded by Elon Musk. It has revolutionized the space industry with reusable rockets and aims to enable human life on Mars.",
    "NASA is the United States government agency responsible for space exploration and research. It has led historic missions like the Apollo moon landings and is currently involved in developing technologies for future space missions.",
    "Tesla is a leading electric vehicle and clean energy company that focuses on accelerating the transition to sustainable energy. Its innovations include electric cars, solar panels, and energy storage solutions.",
    "Blue Origin is an aerospace company founded by Jeff Bezos, aiming to make space travel more affordable and accessible. It is known for its suborbital space tourism program and development of reusable rockets.",
    "Boeing is a major American corporation in the aerospace industry, manufacturing commercial airplanes, defense systems, and satellites. Boeing has a long history of contributions to aviation and space exploration.",
    "Virgin Galactic is a space tourism company founded by Richard Branson. It focuses on providing commercial suborbital spaceflights for passengers, aiming to make space travel accessible to more people."
]

In [10]:
from openai import OpenAI


def docs_to_embeddings(docs: list) -> list:
    openai = OpenAI()
    document_embeddings = []
    for doc in docs:
        response = (
            openai.embeddings.create(
                input=doc,
                model="text-embedding-3-small"
            )
            .data[0]
            .embedding
        )
        document_embeddings.append(response)
    return document_embeddings


docs_embeddings = docs_to_embeddings(documents)

🍩 https://wandb.ai/pratikbhande0-personal/rag-baselining/r/call/019416f3-da95-7b40-b1a6-99300384b295
🍩 https://wandb.ai/pratikbhande0-personal/rag-baselining/r/call/019416f3-dcee-7c73-92e3-47a1c69fdda3
🍩 https://wandb.ai/pratikbhande0-personal/rag-baselining/r/call/019416f3-ddc5-7331-8d10-d37daae5e48f
🍩 https://wandb.ai/pratikbhande0-personal/rag-baselining/r/call/019416f3-de95-7b62-9399-bc24b7b8a3bc
🍩 https://wandb.ai/pratikbhande0-personal/rag-baselining/r/call/019416f3-dfef-7ec2-9502-26be2a07149d
🍩 https://wandb.ai/pratikbhande0-personal/rag-baselining/r/call/019416f3-e0e6-74c3-9a0c-2de645d1bd69


In [11]:
import numpy as np


@weave.op()
def get_most_relevant_document(query):

    openai = OpenAI()
    query_embedding = (
        openai.embeddings.create(
            input=query,
            model="text-embedding-3-small"
        )
        .data[0]
        .embedding
    )


    similarities = [
        np.dot(query_embedding, doc_emb)
        / (np.linalg.norm(query_embedding) * np.linalg.norm(doc_emb))
        for doc_emb in docs_embeddings
    ]


    most_relevant_doc_index = np.argmax(similarities)
    return documents[most_relevant_doc_index]

In [12]:
class RAGModel(weave.Model):
    model_name: str = "gpt-4o"


    @weave.op()
    def predict(self, question: str) -> dict:

        context = get_most_relevant_document(question)


        client = OpenAI()
        query = f"""Please answer the questions using only the following context.
If you don't know the answer, answer 'I don't know.'


context:"
```
{context}
```

question:
{question}"""
        response = client.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "user", "content": query},
            ],
            temperature=0.0,
            response_format={"type": "text"},
        )
        answer = response.choices[0].message.content
        return {"answer": answer, "context": context}

In [13]:
model = RAGModel(
    model_name="gpt-4o"
)
model.predict("What is NASA?")

🍩 https://wandb.ai/pratikbhande0-personal/rag-baselining/r/call/019416f4-d21c-7311-863f-9fbf1229fef9


{'answer': 'NASA is the United States government agency responsible for space exploration and research.',
 'context': 'NASA is the United States government agency responsible for space exploration and research. It has led historic missions like the Apollo moon landings and is currently involved in developing technologies for future space missions.'}

In [14]:
questions = [
    {"question": "Who founded SpaceX?"},
    {"question": "Which U.S. government agency is responsible for space exploration?"},
    {"question": "What company is focused on accelerating the transition to sustainable energy?"},
    {"question": "Which company aims to make space travel more affordable and accessible?"},
    {"question": "Which major American corporation is involved in both aviation and space exploration?"},
    {"question": "What company is focused on commercial space tourism?"}
]

In [15]:
import json

@weave.op()
async def context_precision_score(question, model_output):
    context_precision_prompt = """Given the question, answer, and context, verify whether the context helped in reaching the provided answer.
In the JSON output, if the context was useful, output {{verification: 1}}.
If the context was not useful, output {{verification: 0}}.
Output in valid JSON format only.

Context:
```
{context}
```

Answer:
{answer}
"""

    client = OpenAI()
    prompt = context_precision_prompt.format(
        question=question,
        context=model_output["context"],
        answer=model_output["answer"],
    )
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        response_format={ "type": "json_object" }
    )
    response_message = response.choices[0].message
    response = json.loads(response_message.content)
    return {
        "verdict": int(response["verification"]) == 1,
    }

In [16]:
import nest_asyncio
nest_asyncio.apply()

In [17]:
evaluation = weave.Evaluation(
    dataset=questions,
    scorers=[context_precision_score]
)

In [18]:
model = RAGModel(
    model_name="gpt-4o"
)
await evaluation.evaluate(model)

Evaluated 1 of 6 examples

Evaluated 2 of 6 examples

Evaluated 3 of 6 examples

Evaluated 4 of 6 examples

Evaluated 5 of 6 examples

Evaluated 6 of 6 examples

Evaluation summary
{
    'context_precision_score': {'verdict': {'true_count': 5, 'true_fraction': 0.8333333333333334}},
    'model_latency': {'mean': 5.697587490081787}
}

🍩 https://wandb.ai/pratikbhande0-personal/rag-baselining/r/call/019416f5-9094-70a3-a39a-7da0cd49e05b


{'context_precision_score': {'verdict': {'true_count': 5,
   'true_fraction': 0.8333333333333334}},
 'model_latency': {'mean': 5.697587490081787}}

In [19]:
model = RAGModel(
    model_name="gpt-4o-mini"
)
await evaluation.evaluate(model)

Evaluated 1 of 6 examples

Evaluated 2 of 6 examples

Evaluated 3 of 6 examples

Evaluated 4 of 6 examples

Evaluated 5 of 6 examples

Evaluated 6 of 6 examples

Evaluation summary
{
    'context_precision_score': {'verdict': {'true_count': 6, 'true_fraction': 1.0}},
    'model_latency': {'mean': 5.078067262967427}
}

🍩 https://wandb.ai/pratikbhande0-personal/rag-baselining/r/call/019416f5-dd09-7571-bed5-ab0f7520082f


{'context_precision_score': {'verdict': {'true_count': 6,
   'true_fraction': 1.0}},
 'model_latency': {'mean': 5.078067262967427}}